In [1]:
import MDAnalysis as mda
import numpy as np
import os
from utils import *
import warnings 

# Suppress warnings specific to MDAnalysis
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")

c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dictionary of charged protein residues with key being the Amber resname and the value being the net charge of that residue
residue_charge_dict = {'ARG':1,'LYS':1,'ASP':-1,'GLU':-1,'MG':2}

In [10]:
data_dir = '/Users/gbonn/OneDrive - Northwestern University/Bonnanzio Geoffrey/04 Raw Data/5EJ5/dG_tests/'

input_dir = '5EJ5/complexes/'
output_dir = 'size_tests/int1/'
all_sizes = [8]
all_dirs = ['0','1','2','3','4','5','6','11','14','18']
for curr_size in all_sizes:
    for curr_dir in all_dirs:
        if curr_dir in ['4','6','11','16']:
            residue_charge_dict['INI'] = -5
            MM_charge = -12
        else:
            residue_charge_dict['INI'] = -4
            MM_charge = -11
        # read in the system that you are about to model
        complex = mda.Universe(input_dir + curr_dir + '/qm_complex.pdb')
        ini = complex.select_atoms("resname INI")
        # get the carbonyl carbon of the substrate
        aka_atom_dict = get_substrate_aka_indexes(ini)
        C2_index = aka_atom_dict['C2']
        C2_id = ini.atoms[C2_index].index
        C2_atom = complex.select_atoms("index " +  str(C2_id))
        # get the QM atoms and residues
        QM_residues, active_residues, QM_atoms_indexes,active_atoms_indexes,fixed_atoms_indexes = get_atoms_by_distance(complex,2,curr_size,C2_id)

        # calculate the charge of our system
        total_charge = 0
        # get charge of QM region 
        for residue in QM_residues:
            resname = residue.resname
            if resname in residue_charge_dict:
                total_charge += residue_charge_dict[resname]

        # simplify lists to write to file
        QM_list = simplify_integer_list(QM_atoms_indexes)
        active_list = simplify_integer_list(active_atoms_indexes)
        fixed_list = simplify_integer_list(fixed_atoms_indexes)

        QM_residue_indexes = [residue.resid for residue in QM_residues]

        # Define the input and output file paths
        input_file = "template_MM_script.inp"
        # Open the input file and read its contents
        with open(input_file, 'r') as file:
            content = file.read()
        
        # Replace the {} placeholders with the variable values
        content = content.replace("{}", "{" + active_list + "}" , 1)  # First occurrence
        # Add the custom line to the end
        custom_line = "*pdbfile " + str(MM_charge) +" 1 qm_complex.pdb"    
        content = content.replace("*pdbfile", custom_line , 1)  # First occurrence
        content += '\n'

        # Write the modified content to the output file
        output_path = output_dir + curr_dir + "/MM/Active_"+str(curr_size)+'/'
        file_name = "mm_opt.inp"
        output_file = output_path + file_name
        # Check if the directory exists
        if not os.path.exists(output_path):
            print(f"Directory '{output_path}' does not exist. Creating it...")
            os.makedirs(output_path)
        with open(output_file, 'w') as file:
            file.write(content)
        print(f"MM File processed and saved as {output_file}")


        # Define the input and output file paths
        input_file = "template_QMMM_script.inp"    
        # Open the input file and read its contents
        with open(input_file, 'r') as file:
            content = file.read()

        # Replace the {} placeholders with the variable values
        content = content.replace("{}", "{" + QM_list + "}" , 1)  # First occurrence
        content = content.replace("{}", "{" + active_list + "}", 1)  # Second occurrence

        # Add the custom line to the end
        custom_line = "*pdbfile " + str(total_charge) +" 1 mm_opt.pdb\n"
        content += custom_line

        # Write the modified content to the output file
        output_path = output_dir + curr_dir + "/QM/Active_"+str(curr_size)+'/' 
        file_name = "opt.inp"
        output_file = output_path + file_name
        # Check if the directory exists
        if not os.path.exists(output_path):
            print(f"Directory '{output_path}' does not exist. Creating it...")
            os.makedirs(output_path)
        with open(output_file, 'w') as file:
            file.write(content)
        print(f"Complex QM/MM File processed and saved as {output_file}")

        active_residues_resids = [residue.resid for residue in active_residues]
        # read in the system that you are about to model
        receptor = mda.Universe('size_tests/receptor/system_prep/qm_receptor.pdb')
        active_atoms_in_receptor = get_atoms_by_reslist(receptor,active_residues_resids)

        QM_atoms_in_receptor = get_atoms_by_reslist(receptor,[1113,1114])
        # simplify lists to write to file
        receptor_QM_atoms_list = simplify_integer_list(QM_atoms_in_receptor)
        receptor_active_atoms_list = simplify_integer_list(active_atoms_in_receptor)

        # Define the input and output file paths
        input_file = "template_QMMM_script.inp"
        
        # Open the input file and read its contents
        with open(input_file, 'r') as file:
            content = file.read()

        # Replace the {} placeholders with the variable values
        content = content.replace("{}", "{" + receptor_QM_atoms_list + "}" , 1)  # First occurrence
        content = content.replace("{}", "{" + receptor_active_atoms_list + "}", 1)  # Second occurrence

        # Add the custom line to the end
        custom_line = "*pdbfile -3 1 qm_complex.pdb\n"
        content += custom_line

        output_path = "size_tests/receptor/" + "/Active_"+str(curr_size)+'/' 
        file_name = "opt.inp"
        output_file = output_path + file_name
        # Check if the directory exists
        if not os.path.exists(output_path):
            print(f"Directory '{output_path}' does not exist. Creating it...")
            os.makedirs(output_path)
        with open(output_file, 'w') as file:
            file.write(content)
        print(f"Receptor QM/MM File processed and saved as {output_file}")


MM File processed and saved as size_tests/int1/18/MM/Active_6/mm_opt.inp
Complex QM/MM File processed and saved as size_tests/int1/18/QM/Active_6/opt.inp
Directory 'size_tests/receptor//Active_6/' does not exist. Creating it...
Receptor QM/MM File processed and saved as size_tests/receptor//Active_6/opt.inp
MM File processed and saved as size_tests/int1/18/MM/Active_7/mm_opt.inp
Complex QM/MM File processed and saved as size_tests/int1/18/QM/Active_7/opt.inp
Directory 'size_tests/receptor//Active_7/' does not exist. Creating it...
Receptor QM/MM File processed and saved as size_tests/receptor//Active_7/opt.inp
MM File processed and saved as size_tests/int1/18/MM/Active_8/mm_opt.inp
Complex QM/MM File processed and saved as size_tests/int1/18/QM/Active_8/opt.inp
Receptor QM/MM File processed and saved as size_tests/receptor//Active_8/opt.inp
Directory 'size_tests/int1/18/MM/Active_10/' does not exist. Creating it...
MM File processed and saved as size_tests/int1/18/MM/Active_10/mm_opt.i

In [ ]:
active_residues_resids = [residue.resid for residue in active_residues]
# read in the system that you are about to model
receptor = mda.Universe('size_tests/receptor/system_prep/qm_complex.pdb')
active_atoms_in_receptor = get_atoms_by_reslist(receptor,active_residues_resids)

QM_atoms_in_receptor = get_atoms_by_reslist(receptor,[1113,1114])
# simplify lists to write to file
receptor_QM_atoms_list = simplify_integer_list(QM_atoms_in_receptor)
receptor_active_atoms_list = simplify_integer_list(active_atoms_in_receptor)

In [ ]:
# Define the input and output file paths
input_file = "template_QMMM_script.inp"
output_file = "receptor/opt.inp"

# Open the input file and read its contents
with open(input_file, 'r') as file:
    content = file.read()

# Replace the {} placeholders with the variable values
content = content.replace("{}", "{" + receptor_QM_atoms_list + "}" , 1)  # First occurrence
content = content.replace("{}", "{" + receptor_active_atoms_list + "}", 1)  # Second occurrence

# Add the custom line to the end
custom_line = "*pdbfile -3 1 qm_complex.pdb\n"
content += custom_line

# Write the modified content to the output file
with open(output_file, 'w') as file:
    file.write(content)

print(f"File processed and saved as {output_file}")
